In [ ]:
# Zainstaluj bibliotekę openai, jeśli jeszcze jej nie masz
!pip install openai

In [ ]:
import openai
import json
from google.colab import userdata

# Pobierz API key z Colab Secrets
api_key = userdata.get('')
base_url = "" # Zaktualizuj jeśli URL jest inny w dokumentacji

# Inicjalizacja klienta OpenAI z adresem API Ark Labs
try:
    client = openai.OpenAI(api_key=ark_api_key, base_url=ark_base_url)

    # Prosty test połączenia przez próbę listowania modeli (jeśli API to wspiera)
    # lub przez małe zapytanie chatowe
    # Zgodnie z dokumentacją, użyjemy prostego zapytania chatowego jako testu
    print("Testowanie połączenia z API Ark Labs...")

    test_response = client.chat.completions.create(
        model="speakleash/Bielik-11B-v2.6-Instruct",
        temperature=0.2,
        messages=[
            {"role": "user", "content": "Czym jesteś?"}
        ],
        max_tokens=50 # Ograniczamy tokeny dla szybszego testu
    )

    print("Połączenie z API nawiązane pomyślnie (test przez zapytanie chat)!")
    print(f"Odpowiedź testowa: {test_response.choices[0].message.content}")


except Exception as e:
    print(f"Błąd podczas łączenia z API: {e}")

Testowanie połączenia z API Ark Labs...
Połączenie z API nawiązane pomyślnie (test przez zapytanie chat)!
Odpowiedź testowa: Jestem dużym modelem językowym o nazwie Bielik, stworzonym przez SpeakLeash i ACK Cyfronet AGH. Moim głównym zadaniem jest pom


In [ ]:
# Prosty prompt z inputem
user_input_prompt = "Wyciągnij kluczowe informacje z poniższego tekstu i przedstaw je w ustrukturyzowanej formie."

# 600 TOKENÓW prosty przykład

In [ ]:
# Fragment Pana Tadeusza (ok. 600 znaków)
pan_tadeusz_fragment = """
Litwo! Ojczyzno moja! ty jesteś jak zdrowie;
Ile cię trzeba cenić, ten tylko się dowie,
Kto cię stracił. Dziś piękność twą w całej ozdobie
Widzę i opisuję, bo tęsknię po tobie.
Panno Święta, co Jasnej bronisz Częstochowy
I w Ostrej świecisz Bramie! Ty, co gród zamkowy
Nowogródzki ochraniasz z jego wiernym ludem!
Jak mnie dziecko do zdrowia powróciłaś cudem,
(Gdy od płaczącej matki pod Twoję opiekę
Ofiarowany, martwą podniosłem powiekę;
I zaraz mogłem pieszo, do Twych świątyń progu
Iść za wrócone życie podziękować Bogu),
Tak nas powrócisz cudem na Ojczyzny łono.
"""

In [ ]:
# Przykład struktury JSON, którą chcemy uzyskać - UPROSZCZONY
# Zostawiamy tylko kilka przykładowych elementów, aby model miał więcej "przestrzeni"
# na znalezienie innych pojęć z tekstu.
expected_json_structure_simplified = {
    "tytul_fragmentu": "", # Pusty string jako placeholder
    "autor": "Adam Mickiewicz", # Możemy zostawić autora, bo to informacja o tekście, nie z tekstu fragmentu
    "kluczowe_pojecia": [
        {"pojecie": "Litwa", "opis": "Ojczyzna narratora"},
    ],
    "glowny_motyw": "", # Pusty string jako placeholder
    "forma": "" # Pusty string jako placeholder
}

# Połączenie promptu, tekstu i struktury JSON dla modelu
# Dodajemy instrukcje CoT i podkreślamy, że JSON to TYLKO format/szablon
messages_payload = [
    {"role": "system", "content": "Jesteś asystentem pomagającym wyciągać ustrukturyzowane informacje z tekstu w formacie JSON. Twoim zadaniem jest dokładne przeanalizowanie podanego tekstu i wyodrębnienie z niego kluczowych informacji zgodnych ze wskazanym szablonem JSON. NIE powtarzaj wartości z szablonu, chyba że występują one w tekście do analizy. Skup się na wartościach z tekstu."},
    {"role": "user", "content": f"""
Wyciągnij kluczowe informacje z poniższego tekstu i przedstaw je w ustrukturyzowanej formie JSON.

Tekst do analizy:
---
{pan_tadeusz_fragment}
---

Postępuj krok po kroku:
1. Przeczytaj uważnie tekst.
2. Zidentyfikuj tytuł fragmentu, jeśli jest wskazany lub wynika z kontekstu (w tym przypadku to Inwokacja).
3. Określ autora tekstu.
4. Wyszukaj w tekście kluczowe pojęcia (miejsca, osoby, abstrakcyjne idee) i dla każdego z nich stwórz krótki opis w oparciu o tekst.
5. Zidentyfikuj główny motyw lub temat fragmentu.
6. Określ formę literacką fragmentu (np. inwokacja, apostrofa).
7. NIE ZWRACAJ WYJAŚNIENIA, tylko JSON
8. Na koniec, sformatuj wszystkie zebrane informacje w postaci obiektu JSON, używając poniższego szablonu jako WZORCA formatu. Zastąp placeholder'y (puste stringi) i przykładowe wartości rzeczywistymi informacjami wyciągniętymi z TEKSTU.

Szablon formatu JSON (użyj GO TYLKO jako wzorca struktury, wypełnij danymi Z TEKSTU):
---
{json.dumps(expected_json_structure_simplified, indent=2, ensure_ascii=False)}
---
"""}
]

print("Przygotowany payload dla API (messages):")
print(json.dumps(messages_payload, indent=2, ensure_ascii=False)) # ensure_ascii=False dla lepszego wyświetlania

Przygotowany payload dla API (messages):
[
  {
    "role": "system",
    "content": "Jesteś asystentem pomagającym wyciągać ustrukturyzowane informacje z tekstu w formacie JSON. Twoim zadaniem jest dokładne przeanalizowanie podanego tekstu i wyodrębnienie z niego kluczowych informacji zgodnych ze wskazanym szablonem JSON. NIE powtarzaj wartości z szablonu, chyba że występują one w tekście do analizy. Skup się na wartościach z tekstu."
  },
  {
    "role": "user",
    "content": "\nWyciągnij kluczowe informacje z poniższego tekstu i przedstaw je w ustrukturyzowanej formie JSON.\n\nTekst do analizy:\n---\n\nLitwo! Ojczyzno moja! ty jesteś jak zdrowie;\nIle cię trzeba cenić, ten tylko się dowie,\nKto cię stracił. Dziś piękność twą w całej ozdobie\nWidzę i opisuję, bo tęsknię po tobie.\nPanno Święta, co Jasnej bronisz Częstochowy\nI w Ostrej świecisz Bramie! Ty, co gród zamkowy\nNowogródzki ochraniasz z jego wiernym ludem!\nJak mnie dziecko do zdrowia powróciłaś cudem,\n(Gdy od płaczącej m

In [ ]:
# Wysyłanie zapytania do API i odbiór surowego outputu przy użyciu biblioteki openai
try:
    print("Wysyłanie zapytania do API Ark Labs...")

    # api_payload został zdefiniowany w poprzedniej komórce
    # Upewnij się, że model w api_payload jest poprawny dla Ark Labs
    response = client.chat.completions.create(
        model="speakleash/Bielik-11B-v2.6-Instruct",
        temperature=0.2,
        messages=messages_payload,
        # response_format={"type": "json_object"} # Tymczasowo usuwamy, aby zobaczyć surową odpowiedź
    )

    # Zakładam, że odpowiedź jest w standardowym formacie OpenAI, ale chcemy zobaczyć surowy tekst
    # Odpowiedź z client.chat.completions.create nie jest surowym tekstem HTTP, ale obiektem Response
    # Dostęp do treści wiadomości jest przez .choices[0].message.content
    raw_response_content = response.choices[0].message.content

    print("Otrzymany surowy (tekstowy) output od modelu:")
    print(raw_response_content)

    # Zapisz surową odpowiedź do zmiennej, aby użyć jej w następnej komórce
    # Możemy zapisać cały obiekt response lub tylko raw_response_content
    # Zapiszmy cały obiekt response dla pełniejszego obrazu, jeśli będzie potrzebny
    last_api_response = response


except Exception as e:
    print(f"Wystąpił błąd podczas wysyłania zapytania lub przetwarzania odpowiedzi API: {e}")
    last_api_response = None # Ustaw na None w przypadku błędu

Wysyłanie zapytania do API Ark Labs...
Otrzymany surowy (tekstowy) output od modelu:
{
  "tytul_fragmentu": "Inwokacja",
  "autor": "Adam Mickiewicz",
  "kluczowe_pojecia": [
    {
      "pojecie": "Litwa",
      "opis": "Ojczyzna narratora"
    },
    {
      "pojecie": "Panna Święta",
      "opis": "Matka Boska, opiekunka wiernych"
    },
    {
      "pojecie": "Jasna Góra",
      "opis": "Klasztor na Jasnej Górze w Częstochowie"
    },
    {
      "pojecie": "Ostra Brama",
      "opis": "Brama w Wilnie, z obrazem Matki Boskiej"
    },
    {
      "pojecie": "Nowogródek",
      "opis": "Miasto na Litwie, związane z dzieciństwem Mickiewicza"
    }
  ],
  "glowny_motyw": "Tęsknota za ojczyzną i wiara w opiekę Matki Boskiej",
  "forma": "Inwokacja"
}


In [ ]:
# Parsowanie surowego outputu jako JSON
if last_api_response is not None:
    try:
        # Dostęp do treści wiadomości, którą chcemy sparsować
        structured_output_str = last_api_response.choices[0].message.content

        # Usunięcie znaczników bloków kodu markdown, jeśli istnieją
        if structured_output_str.startswith("```json"):
            structured_output_str = structured_output_str[len("```json"):].strip()
        if structured_output_str.endswith("```"):
            structured_output_str = structured_output_str[:-len("```")].strip()

        # Parsowanie odpowiedzi JSON z modelu
        structured_output = json.loads(structured_output_str)

        print("Udało się sparsować output jako JSON:")
        # Wyświetlanie w fajnej, prostej formie (np. w formie JSON z wcięciami)
        print(json.dumps(structured_output, indent=2, ensure_ascii=False)) # ensure_ascii=False pomaga w wyświetlaniu polskich znaków

    except json.JSONDecodeError:
        print("Błąd: Otrzymana odpowiedź nie jest prawidłowym formatem JSON po usunięciu znaczników.")
        print(f"Surowa odpowiedź po próbie czyszczenia: '{structured_output_str}'")
    except AttributeError:
        print("Błąd: Zmienna last_api_response nie zawiera oczekiwanej struktury (np. .choices[0].message.content).")
    except Exception as e:
        print(f"Wystąpił nieoczekiwany błąd podczas parsowania JSON: {e}")
else:
    print("Brak odpowiedzi API do sparsowania. Wystąpił błąd w poprzedniej komórce.")

Udało się sparsować output jako JSON:
{
  "tytul_fragmentu": "Inwokacja",
  "autor": "Adam Mickiewicz",
  "kluczowe_pojecia": [
    {
      "pojecie": "Litwa",
      "opis": "Ojczyzna narratora"
    },
    {
      "pojecie": "Panna Święta",
      "opis": "Matka Boska, opiekunka wiernych"
    },
    {
      "pojecie": "Jasna Góra",
      "opis": "Klasztor na Jasnej Górze w Częstochowie"
    },
    {
      "pojecie": "Ostra Brama",
      "opis": "Brama w Wilnie, z obrazem Matki Boskiej"
    },
    {
      "pojecie": "Nowogródek",
      "opis": "Miasto na Litwie, związane z dzieciństwem Mickiewicza"
    }
  ],
  "glowny_motyw": "Tęsknota za ojczyzną i wiara w opiekę Matki Boskiej",
  "forma": "Inwokacja"
}


# TRUDNIEJSZY PRZYKŁAD

In [ ]:
# Zasymulowany tekst raportu prasowego Ministerstwa Finansów (około 3000 tokenów)
# Ten tekst jest fikcyjny i stworzony wyłącznie na potrzeby przykładu.
simulated_financial_report = """
Warszawa, 25 października 2024 r.

Komunikat prasowy Ministerstwa Finansów

Stabilność Finansów Publicznych Kluczem do Rozwoju: Ministerstwo Finansów przedstawia dane za III kwartał 2024 r. i planowane działania na IV kwartał oraz rok 2025.

Ministerstwo Finansów z optymizmem patrzy na obecną sytuację finansów publicznych w Polsce. Dane za trzeci kwartał 2024 roku potwierdzają stabilny trend wzrostowy dochodów budżetowych i efektywne zarządzanie wydatkami. W porównaniu do analogicznego okresu roku ubiegłego, dochody budżetu państwa wzrosły o 15%, osiągając poziom 550,3 mld PLN. Największy wzrost zanotowano w dochodach z podatku VAT (wzrost o 18%, do 285,1 mld PLN), co świadczy o dynamicznym rozwoju konsumpcji i skuteczności działań uszczelniających system podatkowy. Dochody z podatku CIT również wzrosły o 12% (do 75,9 mld PLN), co odzwierciedla dobrą kondycję polskich przedsiębiorstw. Dochody z podatku PIT wyniosły 95,5 mld PLN, co stanowi wzrost o 10% rok do roku.

Deficyt budżetowy na koniec września 2024 r. ukształtował się na poziomie 45,8 mld PLN, co jest wartością niższą od prognozowanej na ten okres. Planowany deficyt na koniec całego roku 2024 zakłada poziom nieprzekraczający 120 mld PLN. Ministerstwo zakłada, że dzięki kontynuacji obecnej polityki fiskalnej i wzrostowi gospodarczemu, cel ten zostanie osiągnięty.

Dług publiczny, liczony według metodologii UE (EDP), na koniec II kwartału 2024 r. wyniósł 1 500,5 mld PLN, co stanowi 52,1% PKB. Jest to poziom niższy od średniej unijnej i pozostaje w bezpiecznych ramach konstytucyjnych i unijnych. Projekcja długu na koniec 2024 r. zakłada poziom około 54% PKB.

Minister Finansów, Jan Kowalski, skomentował najnowsze dane: "Wyniki za III kwartał 2024 r. pokazują, że polska gospodarka pozostaje odporna na globalne zawirowania, a nasze działania w obszarze finansów publicznych przynoszą wymierne efekty. Jesteśmy na dobrej drodze do osiągnięcia zakładanych celów budżetowych na ten rok."

Wiceminister Finansów, Anna Nowak, dodała: "Szczególnie cieszy nas wysoki wzrost dochodów z VAT. To dowód na skuteczność naszych działań w walce z szarą strefą i dowód na siłę polskiej konsumpcji, która napędza wzrost gospodarczy."

**Planowane działania na IV kwartał 2024 r. i rok 2025:**

Ministerstwo Finansów kontynuuje prace nad pakietem zmian uszczelniających system podatkowy, w szczególności w obszarze VAT i CIT. Planowane jest wprowadzenie nowych narzędzi analitycznych opartych o sztuczną inteligencję w celu lepszego identyfikowania i przeciwdziałania oszustwom podatkowym. Projekt ustawy w tej sprawie jest obecnie na etapie konsultacji publicznych i ma trafić do Sejmu w listopadzie 2024 r. Oczekuje się, że nowe przepisy wejdą w życie od I kwartału 2025 r.

Ponadto, Ministerstwo przygotowuje propozycje zmian w systemie ulg podatkowych dla przedsiębiorców inwestujących w nowe technologie i badania i rozwój (B+R). Celem jest zwiększenie innowacyjności polskiej gospodarki. Szczegóły tych rozwiązań zostaną przedstawione w ramach prac nad budżetem na rok 2025. Konsultacje w tej sprawie planowane są na grudzień 2024 r.

W obszarze wydatków publicznych, priorytetem pozostaje efektywność i inwestycje prorozwojowe. Planowane jest zwiększenie nakładów na cyfryzację administracji skarbowej oraz na projekty infrastrukturalne współfinansowane ze środków unijnych.

Ministerstwo Finansów aktywnie uczestniczy w pracach nad przygotowaniem Krajowego Planu Odbudowy (KPO), współpracując z innymi resortami w celu efektywnego wykorzystania dostępnych środków.

**Analiza Sytuacji Gospodarczej:**

Raport wskazuje również na pozytywne sygnały płynące z polskiej gospodarki. Stopa bezrobocia rejestrowanego na koniec września 2024 r. wyniosła 5,0%, co jest jednym z najniższych wyników w Unii Europejskiej. Prognozy wzrostu PKB na rok 2024 zostały skorygowane w górę do 3,5%, a na rok 2025 prognozowany jest wzrost na poziomie 4,0%. Inflacja, choć nadal podwyższona, wykazuje tendencję spadkową. Według najnowszych danych Głównego Urzędu Statystycznego, inflacja CPI we wrześniu 2024 r. wyniosła 6,5% rok do roku, w porównaniu do 7,0% w sierpniu. Oczekuje się dalszego spadku inflacji w kolejnych miesiącach.

Członek Rady Polityki Pieniężnej, prof. Piotr Zieliński, powiedział w wywiadzie dla agencji Reuters: "Widzimy wyraźne spowolnienie dynamiki wzrostu cen. Działania banku centralnego w połączeniu ze stabilizacją na rynkach surowców przynoszą oczekiwane efekty. W mojej opinii, ścieżka dezinflacji jest trwała."

Ministerstwo Finansów monitoruje sytuację na rynkach finansowych i jest gotowe do podjęcia działań stabilizacyjnych, jeśli będzie to konieczne.

**Podsumowanie:**

Przedstawione dane za III kwartał 2024 r. i plany na przyszłość świadczą o determinacji Ministerstwa Finansów w dążeniu do zapewnienia stabilnych i zrównoważonych finansów publicznych. Wzrost dochodów, kontrolowany deficyt i dług publiczny, a także aktywne działania na rzecz uszczelnienia systemu podatkowego i wspierania inwestycji, tworzą solidne podstawy dla dalszego rozwoju polskiej gospodarki. Ministerstwo Finansów pozostaje w kontakcie z uczestnikami rynku i obywatelami, zapewniając transparentność swoich działań i otwartość na dialog. Dalsze komunikaty dotyczące postępów w realizacji przedstawionych planów będą publikowane regularnie.

Kontakt dla mediów:
Biuro Prasowe Ministerstwa Finansów
Telefon: +48 22 XXX XXXX
Email: press@mf.gov.pl

--- KONIEC KOMUNIKATU ---
"""

# Sprawdzenie przybliżonej liczby tokenów (używając prostego podziału na słowa jako przybliżenie)
# W rzeczywistości, tokenizacja zależy od konkretnego modelu
approx_tokens = len(simulated_financial_report.split())
print(f"Przybliżona liczba tokenów w zasymulowanym tekście: {approx_tokens}")

Przybliżona liczba tokenów w zasymulowanym tekście: 741


In [ ]:
# Docelowa, zagnieżdżona struktura JSON dla trudniejszego przykładu
complex_expected_json_structure = {
  "tytul": "",
  "data_publikacji": "",
  "podmiot": "Ministerstwo Finansów", # Zakładamy stały podmiot
  "zagadnienia_kluczowe": [], # Lista stringów
  "dane_finansowe_i_gospodarcze": [ # Zmieniamy nazwę i dodajemy dane gospodarcze
    {
      "opis": "", # np. Deficyt budżetowy, Wzrost PKB, Inflacja
      "wartosc": None, # Używamy None dla wartości numerycznych
      "jednostka": "", # np. mld PLN, %, rok do roku
      "okres": "" # np. III kwartał 2024, wrzesień 2024, 2025
    }
    # Model powinien wyciągnąć więcej takich obiektów z tekstu
  ],
  "podjete_dzialania": [
    {
      "opis_dzialania": "", # Krótki opis działania
      "szczegoly": "" # Dodatkowe szczegóły, jeśli są w tekście
    }
     # Model powinien wyciągnąć więcej takich obiektów z tekstu
  ],
  "cytaty": [
    {
      "tresc": "",
      "autor": "",
      "stanowisko": ""
    }
    # Model powinien wyciągnąć więcej takich obiektów z tekstu
  ],
  "kontakt_dla_mediow": { # Zagnieżdżony obiekt kontaktowy
      "telefon": "",
      "email": ""
  },
  "podsumowanie_raportu": "" # Krótkie podsumowanie głównych wniosków
}

print("Docelowa struktura JSON (szablon) dla trudniejszego przykładu:")
print(json.dumps(complex_expected_json_structure, indent=2, ensure_ascii=False))

Docelowa struktura JSON (szablon) dla trudniejszego przykładu:
{
  "tytul": "",
  "data_publikacji": "",
  "podmiot": "Ministerstwo Finansów",
  "zagadnienia_kluczowe": [],
  "dane_finansowe_i_gospodarcze": [
    {
      "opis": "",
      "wartosc": null,
      "jednostka": "",
      "okres": ""
    }
  ],
  "podjete_dzialania": [
    {
      "opis_dzialania": "",
      "szczegoly": ""
    }
  ],
  "cytaty": [
    {
      "tresc": "",
      "autor": "",
      "stanowisko": ""
    }
  ],
  "kontakt_dla_mediow": {
    "telefon": "",
    "email": ""
  },
  "podsumowanie_raportu": ""
}


In [ ]:
# Przygotowanie payloadu dla API z nowym tekstem i złożoną strukturą JSON
# Używamy instrukcji CoT i podkreślamy, że JSON to TYLKO format/szablon
complex_messages_payload = [
    {"role": "system", "content": "Jesteś asystentem pomagającym wyciągać ustrukturyzowane informacje z tekstu w formacie JSON. Twoim zadaniem jest dokładne przeanalizowanie podanego tekstu raportu prasowego i wyodrębnienie z niego kluczowych informacji zgodnych ze wskazanym szablonem JSON. NIE powtarzaj wartości z szablonu, chyba że występują one w tekście do analizy (np. 'Ministerstwo Finansów'). Skup się na wartościach wyciągniętych BEZPOŚREDNIO z tekstu."},
    {"role": "user", "content": f"""
Wyciągnij kluczowe informacje z poniższego tekstu raportu prasowego i przedstaw je w ustrukturyzowanej formie JSON.

Tekst do analizy:
---
{simulated_financial_report}
---

Postępuj krok po kroku:
1. Przeczytaj uważnie cały tekst raportu prasowego.
2. Zidentyfikuj tytuł raportu i datę publikacji.
3. Potwierdź podmiot publikujący (powinien być Ministerstwo Finansów).
4. Zidentyfikuj główne zagadnienia poruszane w raporcie i zbierz je w listę.
5. Wyszukaj wszystkie kluczowe dane finansowe i gospodarcze (np. dochody, deficyt, dług publiczny, PKB, inflacja, bezrobocie). Dla każdej danej wyodrębnij jej opis, wartość (spróbuj sparsować na liczbę, jeśli to możliwe, w przeciwnym razie zostaw None lub string), jednostkę (np. mld PLN, %, rok do roku) i okres, którego dotyczy. Zbierz je w listę obiektów.
6. Zidentyfikuj wszystkie zapowiedziane lub podjęte działania Ministerstwa. Dla każdego działania wyodrębnij krótki opis i ewentualne szczegóły. Zbierz je w listę obiektów.
7. Wyszukaj wszystkie cytaty kluczowych osób. Dla każdego cytatu wyodrębnij treść, autora i stanowisko. Zbierz je w listę obiektów.
8. Zidentyfikuj dane kontaktowe dla mediów (telefon i email). Zbierz je w zagnieżdżony obiekt.
9. Stwórz krótkie podsumowanie głównych wniosków z raportu w oparciu o tekst.
10. Na koniec, sformatuj wszystkie zebrane informacje w postaci obiektu JSON, używając poniższego szablonu jako WZORCA formatu. Zastąp placeholder'y (puste stringi, None) i przykładowe wartości rzeczywistymi informacjami wyciągniętymi Z TEKSTU. Upewnij się, że struktura odpowiada szablonowi, w tym zagnieżdżonym listom i obiektom.

Szablon formatu JSON (użyj GO TYLKO jako wzorca struktury, wypełnij danymi Z TEKSTU):
---
{json.dumps(complex_expected_json_structure, indent=2, ensure_ascii=False)}
---
"""}
]

print("Przygotowany payload dla API (messages) dla trudniejszego przykładu:")
print(json.dumps(complex_messages_payload, indent=2, ensure_ascii=False)) # ensure_ascii=False dla lepszego wyświetlania

Przygotowany payload dla API (messages) dla trudniejszego przykładu:
[
  {
    "role": "system",
    "content": "Jesteś asystentem pomagającym wyciągać ustrukturyzowane informacje z tekstu w formacie JSON. Twoim zadaniem jest dokładne przeanalizowanie podanego tekstu raportu prasowego i wyodrębnienie z niego kluczowych informacji zgodnych ze wskazanym szablonem JSON. NIE powtarzaj wartości z szablonu, chyba że występują one w tekście do analizy (np. 'Ministerstwo Finansów'). Skup się na wartościach wyciągniętych BEZPOŚREDNIO z tekstu."
  },
  {
    "role": "user",
    "content": "\nWyciągnij kluczowe informacje z poniższego tekstu raportu prasowego i przedstaw je w ustrukturyzowanej formie JSON.\n\nTekst do analizy:\n---\n\nWarszawa, 25 października 2024 r.\n\nKomunikat prasowy Ministerstwa Finansów\n\nStabilność Finansów Publicznych Kluczem do Rozwoju: Ministerstwo Finansów przedstawia dane za III kwartał 2024 r. i planowane działania na IV kwartał oraz rok 2025.\n\nMinisterstwo Finan

In [ ]:
# Wysyłanie zapytania do API i odbiór surowego outputu przy użyciu biblioteki openai (trudniejszy przykład)
# Używamy klienta zdefiniowanego w komórce 0cd9956c
if 'client' in locals():
    try:
        print("Wysyłanie zapytania do API Ark Labs (trudniejszy przykład)...")

        # complex_messages_payload został zdefiniowany w poprzedniej komórce
        response = client.chat.completions.create(
            model="speakleash/Bielik-11B-v2.6-Instruct", # Zmień na nazwę modelu dostępnego w API Ark Labs, np. "gpt-4o" lub inny z dokumentacji, który najlepiej radzi sobie z JSON
            messages=complex_messages_payload,
            # Możemy spróbować dodać response_format={"type": "json_object"} ponownie,
            # ale jeśli powodowało to błąd, lepiej zobaczyć surową odpowiedź bez tego.
            # response_format={"type": "json_object"}
            temperature=0.2 # Możesz dostosować temperaturę
        )

        # Dostęp do treści wiadomości
        raw_complex_response_content = response.choices[0].message.content

        print("Otrzymany surowy (tekstowy) output od modelu (trudniejszy przykład):")
        print(raw_complex_response_content)

        # Zapisz surową odpowiedź do zmiennej, aby użyć jej w następnej komórce
        last_complex_api_response = response


    except Exception as e:
        print(f"Wystąpił błąd podczas wysyłania zapytania lub przetwarzania odpowiedzi API (trudniejszy przykład): {e}")
        last_complex_api_response = None # Ustaw na None w przypadku błędu
else:
    print("Błąd: Klient API nie został poprawnie zainicjalizowany.")
    last_complex_api_response = None

Wysyłanie zapytania do API Ark Labs (trudniejszy przykład)...
Otrzymany surowy (tekstowy) output od modelu (trudniejszy przykład):
```json
{
  "tytul": "Stabilność Finansów Publicznych Kluczem do Rozwoju: Ministerstwo Finansów przedstawia dane za III kwartał 2024 r. i planowane działania na IV kwartał oraz rok 2025.",
  "data_publikacji": "25 października 2024 r.",
  "podmiot": "Ministerstwo Finansów",
  "zagadnienia_kluczowe": [
    "Stabilność finansów publicznych",
    "Dane za III kwartał 2024 r.",
    "Planowane działania na IV kwartał 2024 r. i rok 2025",
    "Sytuacja gospodarcza w Polsce"
  ],
  "dane_finansowe_i_gospodarcze": [
    {
      "opis": "Dochody budżetu państwa",
      "wartosc": 550.3,
      "jednostka": "mld PLN",
      "okres": "III kwartał 2024 r."
    },
    {
      "opis": "Wzrost dochodów z VAT",
      "wartosc": 18,
      "jednostka": "procent",
      "okres": "III kwartał 2024 r. w porównaniu do roku ubiegłego"
    },
    {
      "opis": "Dochody z VAT",
  

In [ ]:
# Parsowanie surowego outputu jako JSON (trudniejszy przykład)
if last_complex_api_response is not None:
    try:
        # Dostęp do treści wiadomości, którą chcemy sparsować
        complex_structured_output_str = last_complex_api_response.choices[0].message.content

        # Usunięcie znaczników bloków kodu markdown, jeśli istnieją
        if complex_structured_output_str.startswith("```json"):
            complex_structured_output_str = complex_structured_output_str[len("```json"):].strip()
        if complex_structured_output_str.endswith("```"):
            complex_structured_output_str = complex_structured_output_str[:-len("```")].strip()

        # Parsowanie odpowiedzi JSON z modelu
        complex_structured_output = json.loads(complex_structured_output_str)

        print("Udało się sparsować output jako JSON (trudniejszy przykład):")
        # Wyświetlanie w fajnej, prostej formie (np. w formie JSON z wcięciami)
        print(json.dumps(complex_structured_output, indent=2, ensure_ascii=False)) # ensure_ascii=False pomaga w wyświetlaniu polskich znaków

    except json.JSONDecodeError:
        print("Błąd: Otrzymana odpowiedź nie jest prawidłowym formatem JSON po usunięciu znaczników (trudniejszy przykład).")
        print(f"Surowa odpowiedź po próbie czyszczenia: '{complex_structured_output_str}'")
    except AttributeError:
        print("Błąd: Zmienna last_complex_api_response nie zawiera oczekiwanej struktury (np. .choices[0].message.content).")
    except Exception as e:
        print(f"Wystąpił nieoczekiwany błąd podczas parsowania JSON (trudniejszy przykład): {e}")
else:
    print("Brak odpowiedzi API do sparsowania (trudniejszy przykład). Wystąpił błąd w poprzedniej komórce.")

Udało się sparsować output jako JSON (trudniejszy przykład):
{
  "tytul": "Stabilność Finansów Publicznych Kluczem do Rozwoju: Ministerstwo Finansów przedstawia dane za III kwartał 2024 r. i planowane działania na IV kwartał oraz rok 2025.",
  "data_publikacji": "25 października 2024 r.",
  "podmiot": "Ministerstwo Finansów",
  "zagadnienia_kluczowe": [
    "Stabilność finansów publicznych",
    "Dane za III kwartał 2024 r.",
    "Planowane działania na IV kwartał 2024 r. i rok 2025",
    "Sytuacja gospodarcza w Polsce"
  ],
  "dane_finansowe_i_gospodarcze": [
    {
      "opis": "Dochody budżetu państwa",
      "wartosc": 550.3,
      "jednostka": "mld PLN",
      "okres": "III kwartał 2024 r."
    },
    {
      "opis": "Wzrost dochodów z VAT",
      "wartosc": 18,
      "jednostka": "procent",
      "okres": "III kwartał 2024 r. w porównaniu do roku ubiegłego"
    },
    {
      "opis": "Dochody z VAT",
      "wartosc": 285.1,
      "jednostka": "mld PLN",
      "okres": "III kwarta